In [11]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to c:\users\admin\appdata\local\temp\pip-req-build-cngqz1j_
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/11.7 MB ? eta -:--:--
    --------------------------------------- 0.2/11.7 MB 3.9 MB/s eta 0:00:03
    --------------------------------------- 0.3/11.7 MB 3.2 MB/s eta 0:00:04
    --------------------------------------- 0.3/11.7 MB 2.1 MB/s eta 0:00:06
    --------------------------------------- 0.3/11.7 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.3/11.7 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/11.7 MB 1.3 MB/s eta 0:00:09
   - -------------------------------------- 0.4/11.7 MB 1.1 MB/s eta 0:00:10
   - -------------------------------------- 0.4/11.7 MB 1.1 MB/s eta 0:00:11
   - ---

  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-cngqz1j_'


In [42]:
import os
import json
import math
import pandas as pd
import numpy as np
import nltk
import random
import emoji 
import re
import pandasql as ps
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import string
from nltk.metrics import edit_distance
import pke

# 1. Import data

## Define function

In [13]:
# Data reading function
def read_valid_json_files(folder_path):
    json_files = [file for file in os.listdir(folder_path) if file.endswith('.json')]
    valid_data = []
    for json_file in json_files:
        try:
            with open(os.path.join(folder_path, json_file), 'r') as f:
                data = json.load(f)
                for file in data:                    
                    file['tw_id'] = json_file.split('_')[0]
                    file['file_num'] = json_file[-6]
                valid_data.extend(data)
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON file: {json_file}")
    return valid_data

In [23]:
# Data extracting function
def extract_data(data):
    extracted_data = []
    for item in data:
        post_id = item.get("post_id")
        user_id = item.get("user_id")
        text = item.get("text")
        username = item.get("username")
        time = item.get("time")
        timestamp = item.get("timestamp")
        tw_id = item.get("tw_id")
        file_num = item.get("file_num")
        extracted_data.append({"post_id":post_id,
            "username": username, "user_id": user_id, "text": text
                               , "time": time
                               , "timestamp": timestamp
                               ,'tw_id':tw_id.lower(),"file_num": file_num
                               })
    return extracted_data

## Import FB data

In [15]:
folder_path_fb =  r"D:\An\DataFeild\NCKH\Facebook_1800users_raw"
valid_json_data_fb = read_valid_json_files(folder_path_fb)
extracted_data_fb = extract_data(valid_json_data_fb)

Skipping invalid JSON file: taylorswift_facebook_6.json


In [16]:
extracted_data_fb

[{'post_id': '682254716601826',
  'username': 'Ronaldinho Gaúcho',
  'user_id': '100044519105951',
  'text': 'Feliz aniversário pra vc minha mãe... sentimos sua falta todos os dias ❤️\n\nHappy birthday to you my mother... we miss you every day ❤️',
  'time': '2022-09-30 08:15:39',
  'tw_id': '10ronaldinho',
  'file_num': '1'},
 {'post_id': '681088063385158',
  'username': 'Ronaldinho Gaúcho',
  'user_id': '100044519105951',
  'text': "Meu primeiro gol por este gigante que é o AC Milan 🔴⚫\nLogo decidindo o Derby della Madonnina!!! Valeu o presente Kaká assim fica fácil kkkkkk 💪🏾💪🏾\n\nMy first goal for this giant that is AC Milan 🔴⚫\nSoon deciding on the Madonna Derby!!! Thanks for the gift Kaká it's easy like this lol 💪🏾 💪🏾",
  'time': '2022-09-28 20:52:45',
  'tw_id': '10ronaldinho',
  'file_num': '1'},
 {'post_id': '680381473455817',
  'username': 'Ronaldinho Gaúcho',
  'user_id': '100044519105951',
  'text': "Hj é nosso último passo antes da Copa do Mundo… Vamos tds juntos em busca 

In [19]:
# Create a DataFrame from the extracted data
FB = pd.DataFrame(extracted_data_fb)

# Print the first 5 rows
FB

,post_id,username,user_id,text,time,tw_id,file_num
0,682254716601826,Ronaldinho Gaúcho,100044519105951,Feliz aniversário pra vc minha mãe... sentimos...,2022-09-30 08:15:39,10ronaldinho,1
1,681088063385158,Ronaldinho Gaúcho,100044519105951,Meu primeiro gol por este gigante que é o AC M...,2022-09-28 20:52:45,10ronaldinho,1
2,680381473455817,Ronaldinho Gaúcho,100044519105951,Hj é nosso último passo antes da Copa do Mundo...,2022-09-27 22:54:04,10ronaldinho,1
3,679750666852231,Ronaldinho Gaúcho,100044519105951,Once upon a time at AC Milan ✨\n\n#UCL,2022-09-27 02:47:24,10ronaldinho,1
4,676457457181552,Ronaldinho Gaúcho,100044519105951,Parabéns xará… merece tudo de melhor sempre 👏🏾...,2022-09-23 00:08:51,10ronaldinho,1
...,...,...,...,...,...,...,...
716645,230236573706615,Emily Deschanel Fanpage En/Sk,229475857116020,3 days left till Bones s07e01!!! zostávajú 3 d...,2011-10-31 17:39:22,zooeydeschanel,6
716646,229489370448002,Emily Deschanel Fanpage En/Sk,None,Emily Deschanel Fanpage En/Sk added 17 new pho...,2011-10-29 23:06:54,zooeydeschanel,6
716647,229488567114749,Emily Deschanel Fanpage En/Sk,None,Emily Deschanel Fanpage En/Sk added 8 new phot...,2011-10-29 23:04:48,zooeydeschanel,6
716648,229485493781723,Emily Deschanel Fanpage En/Sk,None,Emily Deschanel & David Boreanaz,2011-10-29 22:58:14,zooeydeschanel,6


## Import X data

In [24]:
folder_path_x = r"D:\An\DataFeild\NCKH\Twitter_300users_raw"
valid_json_data_x = read_valid_json_files(folder_path_x)
extracted_data_x = extract_data(valid_json_data_x)

In [25]:
extracted_data_x

[{'post_id': 1594049106615291907,
  'username': 'Bill Gates',
  'user_id': 'billgates',
  'text': 'In order to solve our world’s sanitation crisis, we need smarter toilets that don’t rely on sewage systems and prevent the spread of diseases. #WorldToiletDay https://t.co/gbp9F0GxlE',
  'time': None,
  'timestamp': 1668885897,
  'tw_id': 'raw.json',
  'file_num': 'w'},
 {'post_id': 1593708872291663872,
  'username': 'Bill Gates',
  'user_id': 'billgates',
  'text': 'Collaboration and more open access to data have made seemingly impossible health breakthroughs a reality, like COVID-19 vaccines. @AlzData is putting the power of data behind one of our most critical health challenges. https://t.co/Qtatf2WMds',
  'time': None,
  'timestamp': 1668804779,
  'tw_id': 'raw.json',
  'file_num': 'w'},
 {'post_id': 1593678220645511168,
  'username': 'Bill Gates',
  'user_id': 'billgates',
  'text': 'From decreasing maternal mortality to lowering rates of vaccine-preventable diseases, @WendoAszed is 

In [26]:
# Create a DataFrame from the extracted data
X = pd.DataFrame(extracted_data_x)

X=X[['username','timestamp','user_id','post_id','text']]

# Print the first 5 rows
X

,username,timestamp,user_id,post_id,text
0,Bill Gates,1668885897,billgates,1594049106615291907,In order to solve our world’s sanitation crisi...
1,Bill Gates,1668804779,billgates,1593708872291663872,Collaboration and more open access to data hav...
2,Bill Gates,1668797471,billgates,1593678220645511168,From decreasing maternal mortality to lowering...
3,Bill Gates,1668716833,billgates,1593340000149549056,Africa has always been at the heart of our fou...
4,Bill Gates,1668715519,billgates,1593334489656139779,@Amref_Worldwide is doing great things for hea...
...,...,...,...,...,...
265188,arjun rampal,1504789510,rampalarjun,905778973015191552,Thanks so much. Glad u enjoyed https://t.co/BY...
265189,arjun rampal,1504785333,rampalarjun,905761454619578368,Ty @khaleejtimes for this. The only one who di...
265190,arjun rampal,1504780643,rampalarjun,905741781538660352,Yay u guys are the best https://t.co/eaUrXB0SJo
265191,arjun rampal,1504780493,rampalarjun,905741154427293696,Thanks Pragya darling. https://t.co/3jnzX8u22J


# 2. Data sampling

## Missing value processing

In [27]:
FB.isnull().sum()

post_id       12
username       0
user_id     2874
text           0
time           7
tw_id          0
file_num       0
dtype: int64

In [28]:
FB[FB['user_id'].isnull()]

,post_id,username,user_id,text,time,tw_id,file_num
1564,284431824993765,Ronaldinho: A Legend To Be Remembered,None,This page is back up and running! What do you ...,2012-11-10 00:22:41,10ronaldinho,5
1577,259116220858659,Ronaldinho: A Legend To Be Remembered,None,,2012-08-08 19:52:51,10ronaldinho,5
1579,258910240879257,Ronaldinho: A Legend To Be Remembered,None,,2012-08-07 22:47:14,10ronaldinho,5
4469,31617152043,50 Cent: The Money and The Power,None,Check out the cast of 50 Cent: The Power and T...,2008-10-22 02:21:21,50cent,3
4804,10157465777222127,CS50,None,by Arturo Real,2020-10-29 03:21:21,50cent,4
...,...,...,...,...,...,...,...
716644,231398130257126,Emily Deschanel Fanpage En/Sk,None,The Memories in the Shallow Grave,2011-11-03 06:37:44,zooeydeschanel,6
716646,229489370448002,Emily Deschanel Fanpage En/Sk,None,Emily Deschanel Fanpage En/Sk added 17 new pho...,2011-10-29 23:06:54,zooeydeschanel,6
716647,229488567114749,Emily Deschanel Fanpage En/Sk,None,Emily Deschanel Fanpage En/Sk added 8 new phot...,2011-10-29 23:04:48,zooeydeschanel,6
716648,229485493781723,Emily Deschanel Fanpage En/Sk,None,Emily Deschanel & David Boreanaz,2011-10-29 22:58:14,zooeydeschanel,6


In [29]:
FB = FB.dropna(axis=0)

In [30]:
FB.isnull().sum()

post_id     0
username    0
user_id     0
text        0
time        0
tw_id       0
file_num    0
dtype: int64

In [31]:
X.isnull().sum()

username     0
timestamp    0
user_id      0
post_id      0
text         0
dtype: int64

### FB data sampling

In [33]:
fb_user_id = list(FB['tw_id'].unique())
fb_selected_posts = {}
for user in fb_user_id:
    for file_num in FB[FB['tw_id']==user]['file_num'].unique():
        data = FB[(FB['tw_id']==user) & (FB['file_num'] == file_num)]
        num_posts = len(data)
        if user in fb_selected_posts.keys():
            if num_posts >= 100:            
                fb_selected_posts[user] = fb_selected_posts[user]._append(data.sample(random.randint(50, 100),random_state=50),ignore_index=True)
            else:
                fb_selected_posts[user] = fb_selected_posts[user]._append(data)
        else: 
            if num_posts >= 100:            
                fb_selected_posts[user] = data.sample(random.randint(50, 100),random_state=50)
            else:
                fb_selected_posts[user] = data

In [35]:
data_fb = pd.DataFrame(columns=['post_id','time','username','user_id','text','tw_id'])

In [36]:
for user_id, df in fb_selected_posts.items():
    data_fb = data_fb._append(df, ignore_index=True)

In [37]:
data_fb

,post_id,time,username,user_id,text,tw_id,file_num
0,3036097666487581,2020-06-25 00:58:40,Ronaldinho Gaúcho,142523572511686,🇧🇷 Ronaldinho Gaúcho ➡️ Diego Maradona 🇦🇷\n\nW...,10ronaldinho,1
1,532321134928519,2022-02-28 00:43:54,Ronaldinho Gaúcho,100044519105951,Lembranças de um dia mt especial com o mengão!...,10ronaldinho,1
2,2655139637916721,2020-01-08 03:09:40,Ronaldinho Gaúcho,142523572511686,Com a LBLV Online você amplia seu portfólio de...,10ronaldinho,1
3,565252861635346,2022-04-18 20:19:57,Ronaldinho Gaúcho,100044519105951,Bora galera que hj 12:00 BR tem Reis do Drible...,10ronaldinho,1
4,328093652017936,2021-04-30 22:09:28,Ronaldinho Gaúcho,100044519105951,"Irmão K2, feliz de te receber na minha casa no...",10ronaldinho,1
...,...,...,...,...,...,...,...
108853,248378531892419,2011-12-04 22:38:28,Emily Deschanel Fanpage En/Sk,229475857116020,http://www.youtube.com/watch?v=zz8hO8Lw3gg&fea...,zooeydeschanel,6
108854,236280836435522,2011-11-13 14:21:20,Emily Deschanel Fanpage En/Sk,229475857116020,Have you seen a new episode of Bones? What do ...,zooeydeschanel,6
108855,231990626864543,2011-11-04 15:46:40,Emily Deschanel Fanpage En/Sk,229475857116020,"so, what do you think about new episode? <3 čo...",zooeydeschanel,6
108856,231399870256952,2011-11-03 06:41:10,Emily Deschanel Fanpage En/Sk,229475857116020,just a few hours left till s07e01 - The Memori...,zooeydeschanel,6


### X data sampling

In [38]:
X_user_id = list(X['user_id'].unique())
X_selected_posts = {}
for user in X_user_id:
    num_posts = len(X[X['user_id']==user])
    if num_posts >= 100:
        X_selected_posts[user] = X[X['user_id']==user].sample(random.randint(50, 100),random_state=10)
    else:
        X_selected_posts[user] = X[X['user_id']==user]

In [39]:
data_X = pd.DataFrame(columns=['username','timestamp','user_id','post_id','text'])

In [40]:
for user_id, df in X_selected_posts.items():
    data_X = data_X._append(df, ignore_index=True)

In [41]:
data_X

,username,timestamp,user_id,post_id,text
0,Bill Gates,1570472813,billgates,1181274713559052289,I’m lucky to have known Stephen Hawking — and ...
1,Bill Gates,1568137891,billgates,1171481339893149696,Congratulations @BradSmi and @CarolAnnBrowne! ...
2,Bill Gates,1555786552,billgates,1119676070981038080,Melinda’s new book The #MomentofLift is an urg...
3,Bill Gates,1656694954,billgates,1542916583651037184,I got to join the @Breakthrough team to talk a...
4,Bill Gates,1558268460,billgates,1130085946442100737,"Together, we can achieve what @Rotary first en..."
...,...,...,...,...,...
20693,arjun rampal,1560172156,rampalarjun,1138070625350705157,What an incredible champion you are @YUVSTRONG...
20694,arjun rampal,1649913525,rampalarjun,1514473212175867906,Thank you Taran 🙏🏽 https://t.co/EOuqjRPgyy
20695,arjun rampal,1670152410,rampalarjun,1599361246570373122,@nairsameer happy happy birthday onwards and u...
20696,arjun rampal,1516624954,rampalarjun,955420422459322369,Just got my @dabbooratnani @manishadratnani ca...


In [43]:

nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")

stopword=set(stopwords.words('english'))

def clean(text):
    # Convert a string to lowercase
    text = str(text).lower()
    # Remove characters within square bracket
    text = re.sub('\[.*?\]', '', text)
    # Remove HTML tags
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # Remove web URLs
    text = re.sub('<.*?>+', '', text)
    # Remove punctuation marks
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Remove newline characters
    text = re.sub('\n', '', text)
    # Remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)
    # Remove emoji
    text = emoji.replace_emoji(text)
    # Remove stop words that are common words that do not carry much meaning in a text
    text = [word for word in text.split(' ') if word not in stopword]
    # Join the remaining words into a text string
    text = " ".join(text)
    # Apply the stemmer algorithm to reduce words to their root form
    # ví dụ như học -> học, học sinh -> học, học tập -> học. 
    # A stemmer is a tool to reduce the number of different but semantically similar words in a text
    text = [stemmer.stem(word) for word in text.split(' ')]
    # Reconnect stemmed words into a string text
    text = " ".join(text)
    return text

<>:10: SyntaxWarning: invalid escape sequence '\['
<>:12: SyntaxWarning: invalid escape sequence '\S'
<>:20: SyntaxWarning: invalid escape sequence '\w'
<>:10: SyntaxWarning: invalid escape sequence '\['
<>:12: SyntaxWarning: invalid escape sequence '\S'
<>:20: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_9536\1340878721.py:10: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9536\1340878721.py:12: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9536\1340878721.py:20: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
data_fb["text"] = data_fb["text"].apply(clean)

In [45]:
data_X["text"] = data_X["text"].apply(clean)

Merge

# 3. EMBEDDING

In [ ]:
model_e5 = SentenceTransformer('intfloat/e5-small-v2')
#model_gte_large=SentenceTransformer('thenlper/gte-large')

## FB

In [59]:
# Transform the text data into list
fb_text_list = data_fb["text"].tolist()
# Encode the text data into embeddings
embeddings_fb_e5 = model_e5.encode(fb_text_list, convert_to_tensor=True)
# Convert the embeddings tensor to a numpy array
embeddings_fb_e5 = embeddings_fb_e5.cpu().detach().numpy()
# Convert the embeddings numpy array to a list
fb_veclist = embeddings_fb_e5.tolist()
# Add the embeddings list to the DataFrame
data_fb["text_vector"] = fb_veclist

In [60]:
data_fb

,post_id,time,username,user_id,text,tw_id,file_num,text_vector
0,3036097666487581,2020-06-25 00:58:40,Ronaldinho Gaúcho,142523572511686,ronaldinho gaúcho diego maradona cant wait s...,10ronaldinho,1,"[-0.09626282751560211, 0.06659062951803207, 0...."
1,532321134928519,2022-02-28 00:43:54,Ronaldinho Gaúcho,100044519105951,lembrança de um dia mt especi com mengão hj fo...,10ronaldinho,1,"[-0.11741817742586136, 0.06256897002458572, -0..."
2,2655139637916721,2020-01-08 03:09:40,Ronaldinho Gaúcho,142523572511686,com lblv onlin você amplia seu portfólio de in...,10ronaldinho,1,"[-0.08463486284017563, 0.0015895318938419223, ..."
3,565252861635346,2022-04-18 20:19:57,Ronaldinho Gaúcho,100044519105951,bora galera que hj br tem rei dribl com meus ...,10ronaldinho,1,"[-0.12356851249933243, 0.04774034023284912, -0..."
4,328093652017936,2021-04-30 22:09:28,Ronaldinho Gaúcho,100044519105951,irmão feliz de te receb na minha casa brasil ...,10ronaldinho,1,"[-0.05062524229288101, 0.019650930538773537, 0..."
...,...,...,...,...,...,...,...,...
108853,248378531892419,2011-12-04 22:38:28,Emily Deschanel Fanpage En/Sk,229475857116020,,zooeydeschanel,6,"[-0.08708294481039047, 0.00997420959174633, 0...."
108854,236280836435522,2011-11-13 14:21:20,Emily Deschanel Fanpage En/Sk,229475857116020,seen new episod bone think vide ste novú epizó...,zooeydeschanel,6,"[-0.13427789509296417, 0.02153170108795166, 0...."
108855,231990626864543,2011-11-04 15:46:40,Emily Deschanel Fanpage En/Sk,229475857116020,think new episod čo si myslít novej epizód,zooeydeschanel,6,"[-0.1158071979880333, 0.02561568096280098, 0.0..."
108856,231399870256952,2011-11-03 06:41:10,Emily Deschanel Fanpage En/Sk,229475857116020,hour left till memori shallow grave pár h...,zooeydeschanel,6,"[-0.10997337847948074, 0.041750941425561905, 0..."


## X

In [52]:
# Transform the text data into list
x_text_list = data_X["text"].tolist()
# Encode the text data into embeddings
embeddings_x_e5 = model_e5.encode(x_text_list, convert_to_tensor=True)
# Convert the embeddings tensor to a numpy array
embeddings_x_e5 = embeddings_x_e5.cpu().detach().numpy()
# Convert the embeddings numpy array to a list
x_veclist = embeddings_x_e5.tolist()
# Add the embeddings list to the DataFrame
data_X["text_vector"] = x_veclist

In [58]:
data_X

,username,timestamp,user_id,post_id,text,text_vector
0,Bill Gates,1570472813,billgates,1181274713559052289,i'm lucki known stephen hawk — except person i...,"[-0.05213679373264313, 0.017412422224879265, 0..."
1,Bill Gates,1568137891,billgates,1171481339893149696,congratul bradsmi carolannbrown insight never ...,"[-0.054940029978752136, 0.05346568673849106, -..."
2,Bill Gates,1555786552,billgates,1119676070981038080,melinda new book momentoflift urgent call acti...,"[-0.0856705829501152, 0.03633005544543266, 0.0..."
3,Bill Gates,1656694954,billgates,1542916583651037184,got join breakthrough team talk incred work th...,"[-0.10496671497821808, 0.005237508565187454, 0..."
4,Bill Gates,1558268460,billgates,1130085946442100737,togeth achiev rotari first envis world child s...,"[-0.0912962481379509, 0.05323803797364235, 0.0..."
...,...,...,...,...,...,...
20693,arjun rampal,1560172156,rampalarjun,1138070625350705157,incred champion field inspir mani thank great...,"[-0.07346107810735703, 0.044107113033533096, 0..."
20694,arjun rampal,1649913525,rampalarjun,1514473212175867906,thank taran,"[-0.11442681401968002, 0.05292633920907974, 0...."
20695,arjun rampal,1670152410,rampalarjun,1599361246570373122,nairsam happi happi birthday onward upward gre...,"[-0.061973072588443756, 0.0579402856528759, 0...."
20696,arjun rampal,1516624954,rampalarjun,955420422459322369,got dabbooratnani manishadratnani calendar sho...,"[-0.10415653884410858, 0.05719790607690811, 0...."


# Calculate the distance between two vectors

In [61]:
pip instal scipy

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "instal" - maybe you meant "install"



In [63]:
from scipy.spatial.distance import cdist

In [77]:
vectors_x = np.vstack(data_X['text_vector'].values)

In [78]:
vectors_fb_50000 = np.vstack(data_fb['text_vector'][0:50000].values)

In [79]:
distance_matrix = cdist(vectors_fb_50000, vectors_x, metric='euclidean')

In [80]:
vectors_fb_rest = np.vstack(data_fb['text_vector'][50000:].values)

In [81]:
distance_matrix_rest = cdist(vectors_fb_rest, vectors_x, metric='euclidean')

MemoryError: Unable to allocate 9.08 GiB for an array with shape (58858, 20698) and data type float64

In [72]:
distance_df = pd.DataFrame(distance_matrix)

# 4. Edit distance

In [43]:
# Get 1 username from twitter
def calculate_edit_distance(row):
    return edit_distance(row['fb_username'], row['x_username'])

## Edit distance

In [9]:
#
X_check_edit = X[[
    'user_id','username'
    ]].drop_duplicates(subset=[
        'user_id','username'
        ]).reset_index()
X_check_edit

,index,user_id,username
0,0,billgates,Bill Gates
1,1001,carmeloanthony,Carmelo Anthony
2,2002,nickiminaj,Nicki Minaj
3,3003,oprah,Oprah Winfrey
4,4004,beyonce,BEYONCÉ
...,...,...,...
270,260188,itsgabrielleu,Gabrielle Union
271,261189,shawnmichaels,Shawn Michaels
272,262190,rainnwilson,RainnWilson
273,263191,kerihilson,Keri Hilson


In [10]:
FB_check_edit = FB[[
    'user_id','username','tw_id','file_num'
    ]].drop_duplicates(subset=[
        'user_id','username','tw_id','file_num'
        ])
FB_check_edit

,index,user_id,username,tw_id,file_num
0,0,100044519105951,Ronaldinho Gaúcho,10ronaldinho,1
1,442,142523572511686,Ronaldinho Gaúcho,10ronaldinho,1
2,600,100079039737335,Raphinha,10ronaldinho,2
3,603,102366615733336,Raphinha,10ronaldinho,2
4,1026,114824906744797,Brazil ; The Home Of Legends,10ronaldinho,3
...,...,...,...,...,...
2987,715433,None,I Love you Zooey Deschanel,zooeydeschanel,4
2988,715928,100046379807458,Sarah Nelson Makeup,zooeydeschanel,5
2989,715933,262063347161165,Sarah Nelson Makeup,zooeydeschanel,5
2990,716528,229475857116020,Emily Deschanel Fanpage En/Sk,zooeydeschanel,6


In [11]:
df_edit = pd.DataFrame(columns=['fb_userid','fb_username','file_num','X_username','X_userid','edit_distance'])
for x in range(0,len(X_check_edit)):
    for fb in range(0,len(FB_check_edit)):
        if (X_check_edit['user_id'][x]==FB_check_edit['tw_id'][fb]):
            e = edit_distance(FB_check_edit['username'][fb]
                          ,X_check_edit['username'][x]) 
            df_edit = df_edit.append({
                                'fb_userid':FB_check_edit['user_id'][fb],
                                'fb_username': FB_check_edit['username'][fb],   
                                'file_num' : FB_check_edit['file_num'][fb],
                                'X_username': X_check_edit['username'][x],
                                'X_userid': X_check_edit['user_id'][x],
                                'edit_distance': e
                            }, ignore_index=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18144\2344696927.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_edit = df_edit.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18144\2344696927.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_edit = df_edit.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18144\2344696927.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_edit = df_edit.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18144\2344696927.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_edit = df_edit.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18144\2344696927.py:7: FutureWarning: Th

In [12]:
df_edit

,fb_userid,fb_username,file_num,X_username,X_userid,edit_distance
0,100044428239741,Bill Gates,1,Bill Gates,billgates,0
1,216311481960,Bill Gates,1,Bill Gates,billgates,0
2,None,Bill Gates,1,Bill Gates,billgates,0
3,100063519408243,Think Like A Billionaire,2,Bill Gates,billgates,19
4,104550867610342,Think Like A Billionaire,2,Bill Gates,billgates,19
...,...,...,...,...,...,...
2742,343771889053165,Fans of Arjun rampal,3,arjun rampal,rampalarjun,8
2743,1916298745309122,Arjun Rampal's page,4,arjun rampal,rampalarjun,9
2744,100043876162121,Kangana Ranaut,5,arjun rampal,rampalarjun,10
2745,191103734290916,Arjun Rampal Fans Club,6,arjun rampal,rampalarjun,12


In [13]:
edit_result=ps.sqldf("""
    WITH N AS (
    SELECT *,RANK() OVER (PARTITION BY X_username
                                ORDER BY edit_distance ASC) RANKING
    FROM df_edit
    )
    SELECT X_username,X_userid, fb_username,fb_userid, file_num, edit_distance       
    FROM N         
    WHERE RANKING =1
""" , locals())

In [20]:
edit_result

,X_username,X_userid,fb_username,fb_userid,file_num,edit_distance
0,50cent,50cent,50 Cent,100050432532195,1,2
1,@jason,jason,Jason Mraz,100044147633973,1,7
2,@jason,jason,Jason Mraz,None,1,7
3,@jason,jason,Jason Mraz,6135205697,1,7
4,@jason,jason,Yousician,245680685625507,4,7
...,...,...,...,...,...,...
691,zayn,zaynmalik,Zayn,134251863823674,5,1
692,zooey deschanel,zooeydeschanel,Zooey Deschanel,100044610931410,1,2
693,zooey deschanel,zooeydeschanel,Zooey Deschanel,175644685809528,1,2
694,👼🏼,troyesivan,Conan Gray,100044535785332,5,10


In [25]:
B=ps.sqldf("""
    SELECT X_userid, count(*)
    FROM edit_result
    WHERE file_num=1
    GROUP BY X_userid
    HAVING count(*) >1
""" , locals())
B

,X_userid,count(*)
0,actuallynph,2
1,adamlevine,2
2,adele,3
3,akshaykumar,2
4,aliaa08,2
...,...,...
204,zacefron,3
205,zaynmalik,2
206,zedd,2
207,zendaya,2


In [16]:
true_label=edit_result[edit_result['file_num']=='1']

In [17]:
true_label

,X_username,X_userid,fb_username,fb_userid,file_num,edit_distance
0,50cent,50cent,50 Cent,100050432532195,1,2
1,@jason,jason,Jason Mraz,100044147633973,1,7
2,@jason,jason,Jason Mraz,None,1,7
3,@jason,jason,Jason Mraz,6135205697,1,7
5,A Lord,lord,Lord Sugar,100044170432716,1,8
...,...,...,...,...,...,...
687,ye,kanyewest,Kanye West,101242849169900,1,8
688,zayn,zaynmalik,Zayn,411766845682422,1,1
689,zayn,zaynmalik,Zayn,None,1,1
692,zooey deschanel,zooeydeschanel,Zooey Deschanel,100044610931410,1,2


In [18]:
accuracy=len(true_label['X_username'].unique())/len(X['user_id'].unique())
accuracy

0.8981818181818182

# MODEL

In [77]:

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator



In [88]:
df_cosine = pd.DataFrame(columns=['fb_username','fb_user_id','X_username','X_user_id','cosine','edit_distance'])

In [36]:
data_fb['username'].unique()

array(['Ronaldinho Gaúcho', 'Raphinha', 'Brazil ; The Home Of Legends',
       ..., 'I Love you Zooey Deschanel', 'Sarah Nelson Makeup',
       'Emily Deschanel Fanpage En/Sk'], dtype=object)

In [110]:
df_cosine.to_csv('df_cosine.csv', index=False)

In [107]:
a=df_cosine.groupby(['fb_user_id','X_user_id'])['cosine'].max()

In [37]:
data_fb

,username,user_id,text,file_name
0,Ronaldinho Gaúcho,100044519105951,feliz aniversário pra vc minha mãe sentimo sua...,10Ronaldinho_facebook_1.json
1,Ronaldinho Gaúcho,100044519105951,meu primeiro gol por est gigant que é ac milan...,10Ronaldinho_facebook_1.json
2,Ronaldinho Gaúcho,100044519105951,hj é nosso último passo ant da copa mundo… vam...,10Ronaldinho_facebook_1.json
3,Ronaldinho Gaúcho,100044519105951,upon time ac milan ucl,10Ronaldinho_facebook_1.json
4,Ronaldinho Gaúcho,100044519105951,parabén xará… merec tudo de melhor sempr e qu...,10Ronaldinho_facebook_1.json
...,...,...,...,...
716645,Emily Deschanel Fanpage En/Sk,229475857116020,day left till bone zostávajú dni,ZooeyDeschanel_facebook_6.json
716646,Emily Deschanel Fanpage En/Sk,229475857116020,emili deschanel fanpag ensk ad new photosocto...,ZooeyDeschanel_facebook_6.json
716647,Emily Deschanel Fanpage En/Sk,229475857116020,emili deschanel fanpag ensk ad new photosocto...,ZooeyDeschanel_facebook_6.json
716648,Emily Deschanel Fanpage En/Sk,229475857116020,emili deschanel david boreanaz,ZooeyDeschanel_facebook_6.json


## Extract fb keywords

In [15]:
# Tạo dataframe ví dụ
df_fb_kw = data_fb[['user_id','username','text', 'file_name']]
#df_fb_kw = pd.DataFrame(df_fb_kw)

# Gộp text theo userid
df_fb_kw = df_fb_kw.groupby(['user_id','username'])['text'].agg(lambda x: ' '.join(x)).reset_index()

In [ ]:
extr_keyw_fb=pd.DataFrame(columns=['fb_user_id','fb_text','fb_text_vector'])
# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

In [17]:
for i in range(0,len(df_fb_kw)):

    # load the content of the document, here document is expected to be a simple
    # test string and preprocessing is carried out using spacy
    extractor.load_document(input=str(df_fb_kw['text'][i]),
                            language='en',
                            normalization='stemming')

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives (i.e. `(Noun|Adj)*`)
    extractor.candidate_selection(pos={"NOUN", "PROPN","ADJ"})

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    ex = extractor.candidate_weighting(threshold=0.74,
                                method="average")
    fb_keyphrase=[]
    fb_score=[]
    for (keyphrase, score) in extractor.get_n_best(n=20, stemming=True):
        fb_keyphrase.append(keyphrase)
        fb_score.append(score)
    text_vector = {}

    for key in range(0,len(fb_keyphrase)):
        text_vector[fb_keyphrase[key]]=fb_score[key]

    extr_keyw_fb = extr_keyw_fb.append({
                    'fb_user_id': df_fb_kw['username'][i],
                    'fb_text': df_fb_kw['text'][i],
                    'fb_text_vector': text_vector
                }, ignore_index=True)
    print(i)


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\123855593.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_fb = extr_keyw_fb.append({


0


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\123855593.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_fb = extr_keyw_fb.append({


1


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\123855593.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_fb = extr_keyw_fb.append({


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\123855593.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_fb = extr_keyw_fb.append({


2311


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\123855593.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_fb = extr_keyw_fb.append({


2312
2313


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\123855593.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_fb = extr_keyw_fb.append({


In [37]:
extr_keyw_fb.to_csv('extr_keyw_fb.csv', index=False)

In [36]:
extr_keyw_fb

,fb_user_id,fb_text,fb_text_vector,fb_user_name
0,719210943,year begin high note thank jaipurjewellerysho...,{'farah khan ali founder': 0.01874804515189308...,Farah Khan Ali
1,100024590681364,shraddhakapoor shraddhakapoorfan shraddhakapo...,"{'aarohi glamour': 0.1584931407619709, 'shradd...",Shraddha kapoor The Adorable Actress 2
2,100024879207187,harri style olivia wild arriv don't worri darl...,{'toronto intern film festiv': 0.1134500430316...,One Direction News
3,100028312309284,streamspaceseptemb pm · rock kevin hart lik...,"{'streamspaceseptemb': 0.03302617293536819, 's...",Skandalous Talk
4,100028409966696,sat mar adam shine light,"{'adam': 0.3962987032710889, 'mar': 0.33709389...",A DAY ON THE GREEN
...,...,...,...,...
2309,9899376497,john cena vs batista alexa bliss vs bayley rel...,"{'wwe': 0.012666786397552032, 'tonight nickelo...",John Cena
2310,9934379006,witch tonight today la let come togeth reth...,{'danc crazi summer tour': 0.15098894966210136...,Miranda Cosgrove
2311,99423386070,still danc congrat gatorsvb advanc round home...,{'florida gator men basketb': 0.07891135153082...,Florida Gators
2312,9964154115,get back speed ride year still need get knee b...,{'star chan tatum zendaya yeti': 0.02267612358...,Channing Tatum


In [44]:
bvv=pd.read_csv(r'C:\Users\Admin\Downloads\extr_keyw_x.csv')

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


## Extract X keywords

In [48]:
# Tạo dataframe ví dụ
df_x_kw = data_X[['user_id','username','text', 'file_name']]
#df_fb_kw = pd.DataFrame(df_fb_kw)

# Gộp text theo userid
df_x_kw = df_x_kw.groupby(['user_id','username'])['text'].agg(lambda x: ' '.join(x)).reset_index()

In [49]:
df_x_kw

,user_id,username,text
0,10ronaldinho,Ronaldinho,mai um rolê bruxo agora com meus parceiro da p...
1,143redangel,Angel Locsin,new vlog drop theangelandneilchannel today s...
2,50cent,50cent,cyber monday sale storewid sale end tonight s...
3,actuallynph,Neil Patrick Harris,tomorrow anniversari wondercad onedercad chee...
4,adamlevine,Adam Levine,ojala maluma therudeboyz offici video stream...
...,...,...,...
270,zacefron,Zac Efron,let fix keep make beauti movi togeth love guy ...
271,zaynmalik,zayn,ask collabor celebr jimihendrix birthday unex...
272,zedd,Zedd,couragejd proud buddi thankyoux transientlab ...
273,zendaya,Zendaya,luxurylaw loeweoffici step away music quit...


In [56]:
extr_keyw_X=pd.DataFrame(columns=['x_user_id','x_user_name','x_text','x_text_vector'])
for i in range(0,len(df_x_kw)):
    # initialize keyphrase extraction model, here TopicRank
    extractor = pke.unsupervised.TopicRank()

    # load the content of the document, here document is expected to be a simple
    # test string and preprocessing is carried out using spacy
    extractor.load_document(input=str(data_X['text'][i]),
                            language='en',
                            normalization='stemming')

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives (i.e. `(Noun|Adj)*`)
    extractor.candidate_selection(pos={"NOUN", "PROPN","ADJ"})

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    ex = extractor.candidate_weighting(threshold=0.74,
                                method="average")

    x_keyphrase=[]
    x_score=[]
    for (keyphrase, score) in extractor.get_n_best(n=20, stemming=True):
        x_keyphrase.append(keyphrase)
        x_score.append(score)
    text_vector = {}

    for key in range(0,len(x_keyphrase)):
        text_vector[x_keyphrase[key]]=x_score[key]

    extr_keyw_X = extr_keyw_X.append({
                    'x_user_id': df_x_kw['user_id'][i],
                    'x_user_name': df_x_kw['username'][i],
                    'x_text': df_x_kw['text'][i],
                    'x_text_vector': text_vector
                }, ignore_index=True)
    print(i)


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\3797057757.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_x = extr_keyw_x.append({


0


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\3797057757.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_x = extr_keyw_x.append({


1


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\3797057757.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_x = extr_keyw_x.append({


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\3797057757.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_x = extr_keyw_x.append({


272


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\3797057757.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_x = extr_keyw_x.append({


273
274


C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\3797057757.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extr_keyw_x = extr_keyw_x.append({


In [155]:
get_cosine(extr_keyw_fb['fb_text_vector'][5],extr_keyw_X['X_text_vector'][7])

0.0

In [38]:
for fb in range(0,len(extr_keyw_fb)):
    for x in range(0,len(extr_keyw_X)):
        vt = 1 - get_cosine(extr_keyw_fb['fb_text_vector'][fb],extr_keyw_X['X_text_vector'][x])
        if vt<1 : print(fb,x,vt)

0 0 1.0
0 1 1.0
0 2 1.0
0 3 1.0
0 4 1.0
0 5 1.0
0 6 1.0
0 7 1.0
0 8 1.0
0 9 1.0
0 10 1.0
0 11 1.0
0 12 1.0
0 13 1.0
0 14 1.0
0 15 1.0
0 16 1.0
0 17 1.0
0 18 1.0
0 19 1.0
0 20 1.0
0 21 1.0
0 22 1.0
0 23 1.0
0 24 1.0
0 25 1.0
0 26 1.0
0 27 1.0
0 28 1.0
0 29 1.0
0 30 1.0
0 31 1.0
0 32 1.0
0 33 1.0
0 34 1.0
0 35 1.0
0 36 1.0
0 37 1.0
0 38 1.0
0 39 1.0
0 40 1.0
0 41 1.0
0 42 1.0
0 43 1.0
0 44 1.0
0 45 1.0
0 46 1.0
0 47 1.0
0 48 1.0
0 49 1.0
0 50 1.0
0 51 1.0
0 52 1.0
0 53 1.0
0 54 1.0
0 55 1.0
0 56 1.0
0 57 1.0
0 58 1.0
0 59 1.0
0 60 1.0
0 61 1.0
0 62 1.0
0 63 1.0
0 64 1.0
0 65 1.0
0 66 1.0
0 67 1.0
0 68 1.0
0 69 1.0
0 70 1.0
0 71 1.0
0 72 1.0
0 73 1.0
0 74 1.0
0 75 1.0
0 76 1.0
0 77 1.0
0 78 1.0
0 79 1.0
0 80 1.0
0 81 1.0
0 82 1.0
0 83 1.0
0 84 1.0
0 85 1.0
0 86 1.0
0 87 1.0
0 88 1.0
0 89 1.0
0 90 1.0
0 91 1.0
0 92 1.0
0 93 1.0
0 94 1.0
0 95 1.0
0 96 1.0
0 97 1.0
0 98 1.0
0 99 1.0
1 0 1.0
1 1 1.0
1 2 1.0
1 3 1.0
1 4 1.0
1 5 1.0
1 6 1.0
1 7 1.0
1 8 1.0
1 9 1.0
1 10 1.0
1 11 1.0
1 12 1.0
1 1

#aaaa

In [42]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import numpy as np

In [51]:
fb_sample=np.array(data_fb['text_embeddings'][0:100],dtype=object)
x_sample=np.array(data_X['text_embeddings'][0:100],dtype=object)

In [67]:
fb_nm = normalize(data_fb['text_embeddings'][1].reshape(1, -1))
x_nm = normalize(data_X['text_embeddings'][1].reshape(1, -1))

In [60]:
1-get_cosine(text_to_vector(data_fb['text'][1]),text_to_vector(data_X['text'][1]))

0.969500285933479

In [63]:
df_fb_grouped = df_x_kw

In [122]:
df_fb_grouped

,user_id,username,text
0,10ronaldinho,Ronaldinho,mai um rolê bruxo agora com meus parceiro da p...
1,143redangel,Angel Locsin,new vlog drop theangelandneilchannel today s...
2,50cent,50cent,cyber monday sale storewid sale end tonight s...
3,actuallynph,Neil Patrick Harris,tomorrow anniversari wondercad onedercad chee...
4,adamlevine,Adam Levine,ojala maluma therudeboyz offici video stream...
...,...,...,...
270,zacefron,Zac Efron,let fix keep make beauti movi togeth love guy ...
271,zaynmalik,zayn,ask collabor celebr jimihendrix birthday unex...
272,zedd,Zedd,couragejd proud buddi thankyoux transientlab ...
273,zendaya,Zendaya,luxurylaw loeweoffici step away music quit...


In [128]:
df_fb_grouped

,user_id,username,text
0,10ronaldinho,Ronaldinho,mai um rolê bruxo agora com meus parceiro da p...
1,143redangel,Angel Locsin,new vlog drop theangelandneilchannel today s...
2,50cent,50cent,cyber monday sale storewid sale end tonight s...
3,actuallynph,Neil Patrick Harris,tomorrow anniversari wondercad onedercad chee...
4,adamlevine,Adam Levine,ojala maluma therudeboyz offici video stream...
...,...,...,...
270,zacefron,Zac Efron,let fix keep make beauti movi togeth love guy ...
271,zaynmalik,zayn,ask collabor celebr jimihendrix birthday unex...
272,zedd,Zedd,couragejd proud buddi thankyoux transientlab ...
273,zendaya,Zendaya,luxurylaw loeweoffici step away music quit...


In [134]:
data_load_doc_fb = pd.DataFrame(columns=['fb_user_id','fb_user_name','fb_text_vector'])
for i in range(0,
    #len(df_fb_grouped)
            10):
    pos = {'NOUN', 'PROPN', 'ADJ'}
    grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
    extractor = pke.unsupervised.PositionRank()
    extractor.load_document(input=str(df_fb_grouped['text'][i]),
                            language='en',
                            normalization='stemming')
    extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=20)
    
    ex = extractor.candidate_weighting(window=10,
                              pos=pos)

    fb_keyphrase=[]
    fb_score=[]
    for (keyphrase, score) in extractor.get_n_best(n=20, stemming=True):
        fb_keyphrase.append(keyphrase)
        fb_score.append(score)

    text_vector = {}
    for key in range(0,len(fb_keyphrase)):
        text_vector[fb_keyphrase[key]]=fb_score[key]

    data_load_doc_fb.append({
                    'fb_user_id': df_fb_grouped['user_id'][i],
                    'fb_user_name': df_fb_grouped['username'][i],
                    'fb_text_vector': text_vector
                },ignore_index=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\2807190414.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_load_doc_fb.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\2807190414.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_load_doc_fb.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\2807190414.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_load_doc_fb.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\2807190414.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_load_doc_fb.append({
C:\Users\Admin\AppData\Local\Temp\ipykernel_18024\2807190414.py:27: FutureWarning: T

In [ ]:
pos = {'NOUN', 'PROPN', 'ADJ'}
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
extractor = pke.unsupervised.PositionRank()
extractor.load_document(input=str(df_fb_grouped['text'][i]),
                            language='en',
                            normalization='stemming')
extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=20)
    
ex = extractor.candidate_weighting(window=10,
                          pos=pos)

In [135]:
data_load_doc_fb

,fb_user_id,fb_user_name,fb_text_vector


In [152]:
data_kw_fb = []
for i in range(0,len(df_fb_grouped)):
    pos = {'NOUN', 'PROPN', 'ADJ'}
    grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
    extractor = pke.unsupervised.PositionRank()
    extractor.load_document(input=str(df_fb_grouped['text'][i]),
                            language='en',
                            normalization='stemming')
    extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=10)
    
    ex = extractor.candidate_weighting(window=10,
                              pos=pos)
    keyphrases = extractor.get_n_best(n=10)

    fb_keyphrase=[]
    fb_score=[]
    for (keyphrase, score) in extractor.get_n_best(n=20, stemming=True):
        fb_keyphrase.append(keyphrase)
        fb_score.append(score)
    
    text_vector = {}
    for key in range(0,len(fb_keyphrase)):
        text_vector[fb_keyphrase[key]]=fb_score[key]
    data_kw_fb.append({
                    'fb_text': df_fb_grouped['text'][i],
                    'fb_vector': text_vector
                })
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274


In [ ]:
df_cosine = pd.DataFrame(columns=['fb_username','fb_user_id','X_username','X_user_id','cosine'])
for fb in range(0,len(extr_keyw_fb)):
    for x in range(0,len(extr_keyw_x)):
        vt = 1 - get_cosine(extr_keyw_fb['fb_text_vector'][fb],extr_keyw_x['x_text_vector'][x])
        if vt < 1 :
            df_cosine = df_cosine.append({
                    'fb_username': extr_keyw_fb['fb_user_name'][fb],
                    'fb_user_id' : extr_keyw_fb['fb_user_id'][fb],
                    'X_username': extr_keyw_x['x_user_name'][x],
                    'X_user_id' : extr_keyw_x['x_user_id'][x],
                    'cosine': vt
                }, ignore_index=True)
    print(fb)

In [155]:
data_kw_fb.to_csv('data_kw_fb.csv', index=False)

,fb_text,fb_vector
0,mai um rolê bruxo agora com meus parceiro da p...,{'e jogar com ídolo e grand amigo em doi': 0.0...
1,new vlog drop theangelandneilchannel today s...,{'thank sa mga nakipagchikahan sa amin sa pina...
2,cyber monday sale storewid sale end tonight s...,{'lol green light gang bransoncognac lechemind...
3,tomorrow anniversari wondercad onedercad chee...,{'countri first newslett drop tomorrow wonderc...
4,ojala maluma therudeboyz offici video stream...,{'iamdamienmu vote vote vote tonight dial': 0....
...,...,...
270,let fix keep make beauti movi togeth love guy ...,{'beauti movi togeth love guy': 0.052588302013...
271,ask collabor celebr jimihendrix birthday unex...,{'memyselfandi bestcoversong iheartaward thank...
272,couragejd proud buddi thankyoux transientlab ...,{'couragejd love u buddi happi birthday': 0.06...
273,luxurylaw loeweoffici step away music quit...,{'brunomar luxurylaw tommyhilfig thank man app...


In [72]:
len(FB['tw_id'].unique())

302

In [47]:
i=0
for x in X['user_id'].unique():
    for fb in FB['tw_id'].unique():
        edit=edit_distance(x,fb)
        if edit==0: 
            i+=1
            print(i,x,fb)

1 billgates billgates
2 carmeloanthony carmeloanthony
3 nickiminaj nickiminaj
4 oprah oprah
5 beyonce beyonce
6 mariahcarey mariahcarey
7 zaynmalik zaynmalik
8 rustyrockets rustyrockets
9 rafaelnadal rafaelnadal
10 sarahksilverman sarahksilverman
11 tip tip
12 carlyraejepsen carlyraejepsen
13 floydmayweather floydmayweather
14 maddow maddow
15 sonamakapoor sonamakapoor
16 lewishamilton lewishamilton
17 realhughjackman realhughjackman
18 chelseahandler chelseahandler
19 kanyewest kanyewest
20 mcuban mcuban
21 caradelevingne caradelevingne
22 piersmorgan piersmorgan
23 brunomars brunomars
24 tomhanks tomhanks
25 neymarjr neymarjr
26 thefarahkhan thefarahkhan
27 justinbieber justinbieber
28 wizkhalifa wizkhalifa
29 50cent 50cent
30 edsheeran edsheeran
31 virendersehwag virendersehwag
32 pitbull pitbull
33 samuelljackson samuelljackson
34 kaka kaka
35 waynerooney waynerooney
36 britneyspears britneyspears
37 davidguetta davidguetta
38 liltunechi liltunechi
39 sardesairajdeep sardesairajdee